# Exercise 04: Bayesian Inverse Problems

In the following notebook ...

## Learning objectives
* ...

## Table of contents
* ...

References if any..

In [ ]:
import numpy as np
import cuqi

%load_ext autoreload
%autoreload 2

In [ ]:
model, data, probInfo = cuqi.testproblem.Deconvolution.get_components()
n = model.range_dim
probInfo

In [ ]:
likelihood = cuqi.distribution.Gaussian(model,0.05)

## ML estimation

In [ ]:
def likelihood_function(x):
    return -likelihood(x=x).logpdf(data)

ML = cuqi.solver.minimize(likelihood_function,np.zeros(n))
x_sol = ML.solve()[0]
x_sol = cuqi.samples.CUQIarray(x_sol,geometry=cuqi.geometry.Continuous1D(n))
x_sol.plot()

## MAP estimation

In [ ]:
#prior = cuqi.distribution.Gaussian(np.zeros(n),0.1)
prior = cuqi.distribution.Gaussian(np.zeros(n),0.1*np.ones(n))

In [ ]:
def posterior_function(x):
    return -likelihood(x=x).logpdf(data)-prior.logpdf(x)

MAP = cuqi.solver.minimize(posterior_function,np.zeros(n))
x_map = MAP.solve()[0]
x_map = cuqi.samples.CUQIarray(x_map,geometry=cuqi.geometry.Continuous1D(n))
x_map.plot()

## Posterior sampling

In [ ]:
post = cuqi.distribution.Posterior(likelihood,prior,data)

In [ ]:
sampler = cuqi.sampler.pCN(post,scale=0.1)
samples,_,_ = sampler.sample_adapt(10000,0)

In [ ]:
samples.plot_chain(60)

In [ ]:
samples.burnthin(6000,5).plot_ci(95)

## Posterior sampling (another better sampler)

In [ ]:
sampler = cuqi.sampler.Linear_RTO(likelihood,prior,model,data,np.zeros(n))
samples = sampler.sample(10000,0)

In [ ]:
samples.plot_chain(60)

In [ ]:
samples.burnthin(1000).plot_ci(95)

## High-level interface

In [ ]:
BP = cuqi.problem.BayesianProblem(likelihood,prior,data)

In [ ]:
samples = BP.sample_posterior(5000)

In [ ]:
samples.plot_ci(95,exact=probInfo.exactSolution)

In [ ]:
x_sol = BP.ML()[0]
x_sol = cuqi.samples.CUQIarray(x_sol,geometry=cuqi.geometry.Continuous1D(n))
x_sol.plot()

In [ ]:
x_map = BP.MAP()
x_map.plot()